# Automated facular regions detection based on morphological operators
This notebook consists of an algorithm based on morphological operators to segment facular regions on CaII K3 images obtained by the spectroheliograph of Coimbra Observatory (OGAUC) .... (and the Solar Dynamics Observatory mission (SDO).)

The set available to test the algorithm consists of (two pairs of images of these sensors from the same day). For that, use each of the following names as 'img_in':

2011apr01_K3_OGAUC...

Some of the parameters of the algorithm may need to be tuned in each image for obtaining a better result.


## 0. Preparation 
Importing the necessary modules and defining the size of the window to display the images.

In [ ]:
import diplib as dip
import matplotlib as plt
plt.rcParams["figure.figsize"] = (12,10)

## 1. Reading and displaying the input image 
To test the algorithm with one of the 4 images mentioned above, just insert the name of the file in 'img_in': 

In [ ]:
img_in = '2011apr01_K3_OGAUC'
img = dip.ImageReadTIFF(img_in)
img.Show()

## 2. Creating the sun mask
Pre-processing to filter the white text at the top and bottom of the image using an opening with a structuring element disk of size 'se1':

In [ ]:
se1 = 5
ope = dip.Opening(img,se1)
ope.Show()

Thresholding of the opened image to obtain a binary mask of the sun (note the higher value of the threshold since not all background pixels have a zero value):

In [ ]:
bin = dip.FixedThreshold(ope,30)
bin.Show()

Filling the holes (spots that might have the same intensity of the background) with an operator based on geodesic dilation:

Contour of the mask, obtained through the morphological gradient, superimposed to the input image:

In [ ]:
contSun = dip.MorphologicalGradientMagnitude(bin)
img_ovr_sun = dip.Overlay(img,contSun)
img_ovr_sun.Show()

## 3. Facular Regions Segmentation
Ientification of the facular regions with the white tophat transform, obatined with a opening operator with a disk of size 'se2' as structuring element: 

In [ ]:
se2 = 50
WTH = dip.Tophat(ope,se2)
WTH.Show()

The binarization of the sunspots is obtained through a thresholding of the black top hat:

In [ ]:
WTHbin = dip.FixedThreshold(WTH,30)
WTHbin.Show()

Filling the internal regions of the facular regions not binarized by the previous thresholding operation using a morphological transform:

In [ ]:
WTHbinFH = dip.FillHoles(WTHbin)
WTHbinFH.Show()

Filtering by opening-reconstruction to remove small objects:

In [ ]:
se3 = 3
fWTHbinFH = dip.OpeningByReconstruction(WTHbinFH,se3)
fWTHbinFH.Show()

Determination of the contours of the sunspots with a half-gradient (or morphological external gradient), superimposed to the input image:

In [ ]:
contFacReg = dip.Dilation(fWTHbinFH) - fWTHbinFH
img_ovr_FacReg = dip.Overlay(img,contFacReg)
img_ovr_FacReg.Show()

Save the ouput result in a standard TIFF format:

In [ ]:
img_out = img_in + '_spots.tif'
dip.ImageWriteTIFF(img_ovr_FacReg,img_out)